<a href="https://colab.research.google.com/github/kaspergroenbek98/first-first-year-project/blob/master/SklearnNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/kaspergroenbek98/first-first-year-project.git


Cloning into 'first-first-year-project'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 58 (delta 24), reused 19 (delta 7), pack-reused 0
Unpacking objects: 100% (58/58), done.


In [2]:
import numpy as np
import nltk
import csv
import re
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('punkt')
nltk.download('stopwords')
stopwords = set(nltk.corpus.stopwords.words('english'))


# Read the data into a header and a data np.array - the array is then shuffled
with open('first-first-year-project/data.csv', encoding='latin1') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    data = np.array([np.array(line) for line in csv_reader])
header, OGdata = data[0,:], data[1:,:]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
#Create a list of ALL tweets. (Each element in the list is a tweet)
sequence = list()
for i in range(len(OGdata[:,6])):
  sequence.append(OGdata[i,6].strip())

#Using Sklearn, playing around with the CountVectorizer.
vectorizer = CountVectorizer(analyzer="word", lowercase=True, stop_words= "english",min_df= 5, binary= True)
X = vectorizer.fit_transform(sequence)
X.toarray()
print(vectorizer.get_feature_names())
print(vectorizer.vocabulary_.get(u"newyearsresolution"))
print(X.toarray())
print(X.shape)

["#NewYearsResolution :: Read more books, No scrolling FB/checking email b4 breakfast, stay dedicated to PT/yoga to squash my achin' back!", "#NewYearsResolution Finally master @ZJ10 's part of Kitchen Sink", '#NewYearsResolution to stop being so damn perf _ÙÕÐ_ÙÕÉ', 'My #NewYearsResolution is to help my disabled patients discover the emotional and physical therapy from loving a pet. #adoptarescue', '#NewYearsResolution #2015Goals #2015bucketlist continued\x89\x9d_\x95ü\x8f http://t.co/h4P9B7tWjG']
['000', '10', '100', '1024', '1080p', '11', '12', '13', '14', '15', '17', '18', '1920x1080', '1st', '20', '200', '2013', '2014', '2015', '2015goals', '2016', '21', '24', '2k15', '2nd', '30', '365', '40', '50', '5sos', '__ù', '_ù', '_ù_ä', '_ùª', '_ùªî', '_ùó', '_ùô', '_ùô_', '_ùôî', '_ùõ', '_ùõø', 'able', 'abs', 'accept', 'accomplish', 'accomplished', 'account', 'achieve', 'act', 'acting', 'action', 'active', 'actually', 'add', 'adult', 'adventure', 'advice', 'afraid', 'age', 'ago', 'ahead',